In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
items = pd.read_csv('final_project_data/items.csv')
item_categories = pd.read_csv('final_project_data/item_categories.csv')
sales_train = pd.read_csv('final_project_data/sales_train.csv.gz')
sample_submission = pd.read_csv('final_project_data/sample_submission.csv.gz')
shops = pd.read_csv('final_project_data/shops.csv')
test = pd.read_csv('final_project_data/test.csv.gz')

In [4]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [59]:
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].fillna(0).clip(0,20)
sales_train_months = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'item_cnt_day': 'sum'})
sales_train_months = sales_train_months.rename(columns={'item_cnt_day':'item_cnt_month'})
sales_train_months.item_cnt_month = sales_train_months.item_cnt_month.fillna(0).clip(0,20)
sales_train_months.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [50]:
sales_train_months_33 = sales_train_months[sales_train_months['date_block_num'] == 33]
sales_train_months_33.head()

,date_block_num,shop_id,item_id,item_cnt_month
1577593,33,2,31,1.0
1577594,33,2,486,3.0
1577595,33,2,787,1.0
1577596,33,2,794,1.0
1577597,33,2,968,1.0


In [60]:
merged = test.merge(sales_train_months_33, on=['item_id', 'shop_id'], how='left')[['ID', 'item_cnt_month']]

In [61]:
merged.isna().sum()

ID                     0
item_cnt_month    185520
dtype: int64

In [74]:
merged['item_cnt_month'] = merged.item_cnt_month.fillna(0)
submission = merged.set_index('ID')
submission.to_csv('benchmark1.csv')